## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-09-01-17-17 +0000,nyt,What We Know About the Fatal ICE Shooting in M...,https://www.nytimes.com/2026/01/07/us/ice-shoo...
1,2026-01-09-01-16-39 +0000,nyt,Federal Agents Shoot 2 During Traffic Stop in ...,https://www.nytimes.com/2026/01/08/us/politics...
2,2026-01-09-01-16-03 +0000,nyt,More Agents Head to Minnesota as U.S. Takes Ov...,https://www.nytimes.com/2026/01/08/us/minnesot...
3,2026-01-09-01-10-40 +0000,bbc,Long waits for disability benefit claims unacc...,https://www.bbc.com/news/articles/c931klw03yko...
4,2026-01-09-01-09-46 +0000,wapo,"Border Patrol agents shoot two in Portland, Or...",https://www.washingtonpost.com/nation/2026/01/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
77,trump,91
58,venezuela,35
1,ice,34
2,shooting,27
223,new,23


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
39,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...,199
114,2026-01-08-22-07-42 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...,188
100,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...,178
232,2026-01-08-17-23-16 +0000,nyt,"After ICE Shooting, Trump Administration Deplo...",https://www.nytimes.com/2026/01/08/us/politics...,177
54,2026-01-08-23-47-49 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,174


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
39,199,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...
382,113,2026-01-08-06-05-45 +0000,latimes,'Public execution.' 'Trigger-happy.' Californi...,https://www.latimes.com/california/story/2026-...
100,87,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...
183,49,2026-01-08-19-14-24 +0000,nypost,Protests erupt in Iranian capital after exiled...,https://nypost.com/2026/01/08/world-news/prote...
307,43,2026-01-08-12-01-00 +0000,wsj,China Warns AI Startups Seeking to Emulate Met...,https://www.wsj.com/tech/china-warns-ai-startu...
274,40,2026-01-08-14-56-31 +0000,nypost,‘CBS Evening News’ No. 2 producer fired during...,https://nypost.com/2026/01/08/media/cbs-evenin...
400,40,2026-01-08-03-28-00 +0000,wsj,Trump Team Works Up Sweeping Plan to Control V...,https://www.wsj.com/business/energy-oil/trump-...
346,38,2026-01-08-10-30-00 +0000,wsj,Some mail won’t be postmarked the day you drop...,https://www.wsj.com/politics/policy/a-letter-w...
389,37,2026-01-08-04-25-00 +0000,wsj,"Rep. Steny Hoyer of Maryland, a longtime membe...",https://www.wsj.com/politics/policy/steny-hoye...
61,37,2026-01-08-23-32-43 +0000,nypost,Federal prosecutor calls Newsom ‘king of fraud...,https://nypost.com/2026/01/08/us-news/bill-ess...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
